In [2]:
!pip install import-ipynb
import import_ipynb
import util as u
import pandas as pd

Defaulting to user installation because normal site-packages is not writeable
importing Jupyter notebook from util.ipynb


In [2]:
def get_emendas_materia(codigo_materia):
    url = "https://legis.senado.leg.br/dadosabertos/materia/emendas/%d" % codigo_materia    
    df = u.parse_XML(url,["AutoriaEmenda/Autor/IdentificacaoParlamentar/CodigoParlamentar", 'DataApresentacao'],"Materia/Emendas/Emenda")
    df.rename(columns = {'AutoriaEmenda/Autor/IdentificacaoParlamentar/CodigoParlamentar' : 'CodigoParlamentar'}, inplace = True)        
    df['DataApresentacao'] = pd.to_datetime(df['DataApresentacao'])
    df['CodigoMateria'] = codigo_materia
    df = df.set_index('CodigoMateria')
    return df

In [4]:
df_detalhes_materias = pd.read_csv('df_detalhes_materias.csv')
df_materias_pesquisar = pd.DataFrame(df_detalhes_materias)
df_materias_pesquisar['PegouDetalhe'] = False
df_emendas = pd.DataFrame()    

In [16]:
materias_aux = df_materias_pesquisar.loc[df_materias_pesquisar['PegouDetalhe'] == False]
for codigo, row in materias_aux.iterrows():    
    codigo_materia = row['CodigoMateria']    
    df_emendas_da_materia = get_emendas_materia(int(codigo_materia))    
    df_emendas = pd.concat([df_emendas,df_emendas_da_materia])    
    df_materias_pesquisar.at[codigo, 'PegouDetalhe'] = True    

In [17]:
len(df_materias_pesquisar.loc[df_materias_pesquisar['PegouDetalhe'] == False])

0

In [8]:
len(df_materias_pesquisar.loc[df_materias_pesquisar['PegouDetalhe'] == True])

8125

In [19]:
len(df_emendas)

40330

In [21]:
df_emendas.to_csv('df_emendas.csv')

In [20]:
df_emendas

,CodigoParlamentar,DataApresentacao
CodigoMateria,,
73682,3393,2007-04-16
73682,3399,2009-05-21
80801,4877,2011-08-23
96399,5895,2022-06-21
99517,1173,2019-08-20
...,...,...
160970,6027,2023-11-13
161000,None,2023-11-09
161003,None,2023-11-09


## Limpando as emendas pra ficar só com as dos senadores em questão

In [4]:
df_emendas = pd.read_csv('df_emendas.csv')
df_senadores = pd.read_csv('df_senadores.csv')

In [6]:
len(df_emendas)

40330

In [7]:
df_emendas = df_emendas.merge(df_senadores, on='CodigoParlamentar', how='inner')

In [8]:
len(df_emendas)

14410

In [13]:
df_emendas['CodigoParlamentar'] = df_emendas['CodigoParlamentar'].astype(int)

In [14]:
df_emendas.to_csv('df_emendas_filtradas.csv')